In [2]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import suncalc

import dask.dataframe as dd
from pathlib import Path
from tqdm import tqdm
import re

import datetime as dt

import matplotlib.pyplot as plt
from matplotlib import colors
import soundfile as sf
import matplotlib.patches as patches

In [3]:
import sys

sys.path.append("../src")
sys.path.append("../src/activity")

In [4]:
import subsampling as ss
import activity_assembly as actvt
import bout.assembly as bt
import comparison.data_assembly as comp
import comparison.plot as complot
from core import SITE_NAMES

from cli import get_file_paths
import plot
import pipeline

In [20]:
avail = np.arange(0, 120, 2) + 2
reset_3 = avail[np.where((3*60 % avail) == 0)[0]]
reset_4 = avail[np.where((4*60 % avail) == 0)[0]]
reset_6 = avail[np.where((6*60 % avail) == 0)[0]]
reset_12 = avail[np.where((12*60 % avail) == 0)[0]]
reset_24 = avail[np.where((24*60 % avail) == 0)[0]]

In [5]:
reset_3

array([ 6, 12, 18, 30, 36, 60, 90])

In [6]:
reset_4

array([  6,  12,  24,  30,  48,  60, 120])

In [7]:
reset_6

array([  6,  12,  18,  24,  30,  36,  60,  72,  90, 120])

In [8]:
reset_12

array([  6,  12,  18,  24,  30,  36,  48,  60,  72,  90, 120])

In [9]:
reset_24

array([  6,  12,  18,  24,  30,  36,  48,  60,  72,  90,  96, 120])

In [21]:
f = 6
fact_ = reset_12[np.where((reset_12%f) == 0)[0]]
fact_

array([  6,  12,  18,  24,  30,  36,  48,  60,  72,  90, 120])

In [23]:
step = 1/6
step_by = np.arange(0, (2/3)+step, step)
step_by = step_by[step_by>=(1/6)]
step_by

array([0.16666667, 0.33333333, 0.5       , 0.66666667])

In [8]:
cycle_lengths = fact_
percent_ons = step_by
data_params = dict()
data_params["cycle_lengths"] = cycle_lengths
data_params["percent_ons"] = percent_ons
dc_tags = ss.get_list_of_dc_tags(cycle_lengths, percent_ons)
dc_tags

['30of30',
 '1of6',
 '2of6',
 '2of6',
 '2of6',
 '3of6',
 '3of6',
 '4of6',
 '2of12',
 '3of12',
 '4of12',
 '5of12',
 '6of12',
 '7of12',
 '8of12',
 '3of18',
 '4of18',
 '6of18',
 '7of18',
 '9of18',
 '10of18',
 '12of18',
 '4of24',
 '6of24',
 '8of24',
 '10of24',
 '12of24',
 '14of24',
 '16of24',
 '5of30',
 '8of30',
 '10of30',
 '12of30',
 '15of30',
 '17of30',
 '20of30',
 '6of36',
 '9of36',
 '12of36',
 '15of36',
 '18of36',
 '21of36',
 '24of36',
 '8of48',
 '12of48',
 '16of48',
 '20of48',
 '24of48',
 '28of48',
 '32of48',
 '10of60',
 '15of60',
 '20of60',
 '25of60',
 '30of60',
 '35of60',
 '40of60',
 '12of72',
 '18of72',
 '24of72',
 '30of72',
 '36of72',
 '42of72',
 '48of72',
 '15of90',
 '22of90',
 '30of90',
 '38of90',
 '45of90',
 '52of90',
 '60of90',
 '20of120',
 '30of120',
 '40of120',
 '50of120',
 '60of120',
 '70of120',
 '80of120']

In [13]:
site_keys = ['Carp', 'Telephone']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'bout_time_percentage'
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        file_paths = get_file_paths(data_params)

        activity_arr, btp_arr = comp.generate_activity_btp_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 44/44 [08:55<00:00, 12.18s/it]


Carp HF


100%|██████████| 44/44 [02:24<00:00,  3.28s/it]


Telephone LF


100%|██████████| 44/44 [00:43<00:00,  1.01it/s]


Telephone HF


100%|██████████| 44/44 [08:43<00:00, 11.89s/it]


In [14]:
site_keys = ['Carp', 'Telephone']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'call_rate'
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        file_paths = get_file_paths(data_params)

        activity_arr, callrate_arr = comp.generate_activity_call_rate_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 44/44 [5:56:19<00:00, 485.91s/it]   


Carp HF


100%|██████████| 44/44 [1:26:07<00:00, 117.44s/it]  


Telephone LF


100%|██████████| 44/44 [1:47:08<00:00, 146.11s/it]    


Telephone HF


100%|██████████| 44/44 [1:44:54<00:00, 143.07s/it]  


In [15]:
site_keys = ['Carp', 'Telephone']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'activity_index'
data_params['index_time_block_in_secs'] = 5
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        file_paths = get_file_paths(data_params)

        activity_arr, actvtind_arr = comp.generate_activity_index_percent_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 44/44 [02:39<00:00,  3.62s/it]


Carp HF


100%|██████████| 44/44 [00:43<00:00,  1.01it/s]


Telephone LF


100%|██████████| 44/44 [00:26<00:00,  1.67it/s]


Telephone HF


100%|██████████| 44/44 [02:52<00:00,  3.92s/it]
